In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
%matplotlib notebook

In [ ]:
df_train = pd.read_csv("data/data_train.csv")
df_test = pd.read_csv("data/data_test.csv")
df_train['time_entry'] = pd.to_timedelta(df_train['time_entry'])
df_train['time_exit'] = pd.to_timedelta(df_train['time_exit'])
df_test['time_entry'] = pd.to_timedelta(df_test['time_entry'])
df_test['time_exit'] = pd.to_timedelta(df_test['time_exit'])
df_train['duration'] = df_train['time_exit'] - df_train['time_entry']
df_train['duration'] = df_train['duration'].dt.total_seconds()
df_test['duration'] = df_test['time_exit'] - df_test['time_entry']
df_test['duration'] = df_test['duration'].dt.total_seconds()

In [ ]:
df_train['distance'] = df_train.apply(lambda row: (math.sqrt( ((row['x_entry'] - row['x_exit'])**2)+
                                                             ((row['y_entry']- row['y_exit'])**2) )), axis=1)

In [ ]:
print(df_train.shape[0], df_train.hash.nunique(), df_test.shape[0], df_test.hash.nunique())

In [ ]:
print(df_test[df_test['x_exit'].isnull()].time_entry.describe(), 
      df_test[df_test['x_exit'].isnull()].time_exit.describe())

In [ ]:
df = df_train.groupby('hash')
df_list = list(df)
df_2 = df_test.groupby('hash')
df_2_list = list(df_2)

In [ ]:
tt = df_train[(df_train['x_exit'] >= 3750901.5068) & (df_train['x_exit'] <= 3770901.5068) & 
              (df_train['y_exit'] >= -19268905.6133) & (df_train['y_exit'] <= -19208905.6133)]
tt2 = df_test[(df_test['x_exit'] >= 3750901.5068) & (df_test['x_exit'] <= 3770901.5068) & 
              (df_test['y_exit'] >= -19268905.6133) & (df_test['y_exit'] <= -19208905.6133)]

In [ ]:
def plot_trajectory(df_sample):
    rect = patches.Rectangle((3750901.5068,-19208905.6133), (3770901.5068 - 3750901.5068),
                             (- 19268905.6133 + 19208905.6133),
                             linewidth=1,edgecolor='g',fill = False,hatch = '\\\\\\', label = 'city center')
    city = plt.gca().add_patch(rect)
#     for index, row in df_sample.iterrows():
#         arrows = plt.arrow(row['x_entry'], row['y_entry'], row['x_exit'] - row['x_entry'], row['y_exit'] - row['y_entry'],
#                   label ='Trajectory', color = 'b')
    entries = plt.scatter(df_sample.x_entry,df_sample.y_entry, label='Entry point', color = 'b')
    exits = plt.scatter(df_sample.x_exit,df_sample.y_exit, label='Exit point', color = 'r')
    plt.legend(handles=[city, entries, exits])
    plt.grid(True)
    plt.show()

In [ ]:
plot_trajectory(df_2_list[50][1])

In [ ]:
for index, people in enumerate(df_2_list):
    if people[1].shape[0] <=2:
        print(index, ":",people[1].shape[0])

In [ ]:
df_2_list[33332][1]

In [ ]:
df_train.duration.describe()

In [ ]:
df_train[(df_train['duration'] == 0.0) & (df_train['x_entry'] == df_train['x_exit']) &
         (df_train['y_entry'] == df_train['y_exit'])].shape[0]

In [ ]:
df_train[(df_train['duration'] == 0.0) & ((df_train['x_entry'] != df_train['x_exit']) |
         (df_train['y_entry'] != df_train['y_exit']))]

In [ ]:
df_test[(df_test['duration'] == 0.0) & (df_test.x_exit.isnull()) & (df_test.y_exit.isnull())].shape[0]

In [ ]:
df_train['velocity'] = df_train.apply(lambda row: (row['distance']/row['duration'])
                                      if row['duration']!= 0.0 else 0.0, axis=1)

In [ ]:
df_train[(df_train['velocity'] != 0.0) & (df_train['velocity'] < 500)].velocity.describe()

In [ ]:
df_train[(df_train['velocity'] != 0.0)].velocity.describe()

In [ ]:
df_train = df_train.drop(df_train.index[643223])

In [ ]:
380152.000000 - 377930.000000

In [ ]:
df_test[(df_test['duration'] > 0) & (df_test['duration'] < 5000) & (df_test.x_exit.isnull()) & (df_test.y_exit.isnull())].duration.describe()

In [ ]:
plt.scatter(df_train[df_train['velocity'] != 0.0].distance, df_train[df_train['velocity'] != 0.0].duration)

In [ ]:
df_test['label']= -1
df_test['label'] = df_test.apply(lambda row: 1 
                                 if ((row['duration'] == 0.0)&(math.isnan(row['x_exit'])) & (math.isnan(row['y_exit'])) &
                                      (row['x_entry'] >= 3750901.5068) & (row['x_entry'] <= 3770901.5068) & 
                                      (row['y_entry'] >= -19268905.6133) & (row['y_entry'] <= -19208905.6133)) 
                                 else row['label'], axis=1)
df_test['label'] = df_test.apply(lambda row: 0 
                                 if ((row['duration'] == 0.0)&(math.isnan(row['x_exit'])) & (math.isnan(row['y_exit'])) &
                                     (row['label'] == -1))
                                 else row['label'], axis=1)

In [ ]:
df_test[df_test.x_exit.isnull()]['label'].value_counts()

In [ ]:
df_test['label'] = df_test['label'].replace(-1,0)

In [ ]:
df_test[df_test.x_exit.isnull()][['trajectory_id', 'label']].to_csv('output.csv', index=False)